## Notebook to test different codes and functions.

In [1]:
#folium is the library used to visualize maps
import folium
#library to call Open Route Service(ORS)'s client and requests
from openrouteservice import client
import pandas as pd 
import numpy as np 
import plotly as py
import json

#from isochrones import * 

#i like to be able to see all my columns in a pandas dataframe. this would allow it 
pd.set_option("display.max_columns", None)




In [2]:
from config import api_key
clnt = client.Client(key=api_key) 

In [28]:
#loading up train station data. This file contains the csv data for KL train stations. 
file_url = 'resources/data/raw_data/klang_valley_stations.csv'

#Stop ID is unique for every station. So that column will be used for index
test_data = pd.read_csv(file_url)

lines = list(test_data['Route Name'].unique())
maps = []
params_iso = {'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }

In [4]:
lines

['Ampang Line',
 'Seremban Line',
 'Port Klang Line',
 'KLIA Ekspres',
 'Kelana Jaya Line',
 'Skypark Link',
 'KLIA Transit',
 'KL Monorail Line',
 'BRT Sunway Line',
 'MRT Sungai Buloh - Kajang Line',
 'Sri Petaling Line']

In [5]:
#def dictSetup(dataframe):
 #   station_dict = dataframe.to_dict(orient='index')
  #  for name, station in station_dict.items():
   #     station['locations'] = [station['Longitude'],station['Latitude']]
    #return station_dict

In [24]:
#Perform isochrone request and generates a new column in the stations dataframe containing isochrone data for that station.
#this will save the isochrones requested from Open Route Service(ORS)
#ORS will return the isochrones in geoJSON format
def isoGeoJsonRetriever(parameters,stations,client):

    #ORS isochrones API takes input list of coordinates in field called location. This line creates that column
    stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)
    iso_list = []

    for index, row in stations.iterrows():
        print("Retrieving Isochrone of {} station".format(stations.loc[index,'Name']))
        parameters['locations'] = [row.loc['locations']]
        temp_iso = client.isochrones(**parameters)
        temp_iso = json.dumps(temp_iso)
        iso_list.append(temp_iso)
        print("Success")
    
    stations['iso'] = iso_list

    return

In [7]:
def isoVisualizer(maps,stations, map_icon = ""):
    style_function = lambda x: {'color': '#4ef500' if x['properties']['value']<400 else ('#2100f5' if x['properties']['value']<700.0 else '#f50000'),
                                'fillOpacity' : 0.35 if x['properties']['value']<400 else (0.25 if 400.0<x['properties']['value']<700.0 else 0.05),
                                'weight':2,
                                'fillColor' :'#4ef500' if x['properties']['value']<400 else ('#2100f5' if 400.0<x['properties']['value']<700.0 else '#f50000')}
                                                    #('#6234eb' if x['properties']['value']==600.0 else '#6234eb')
    stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)

    for index, row in stations.iterrows():
        folium.features.GeoJson(row['iso'],style_function = style_function).add_to(maps) # Add GeoJson to map
        if map_icon!="":
            folium.map.Marker(list(reversed(row['locations'])), # reverse coords due to weird folium lat/lon syntax
                                icon=folium.Icon(color='lightgray',
                                            icon_color='#cc0000',
                                            icon=map_icon,
                                            prefix='fa',
                                                ),
                            popup=row['Name'],
                            ).add_to(maps) # Add apartment locations to map      
    print("Done!")

In [8]:
def toMapORS(data,line,params_iso,client):
    # Set up folium map
    if not line in data.values:
        print('{} is not in data frame'.format(line))
        temp = data['Route Name'].unique()
        print('Choose from the following: ')
        print(temp)
        return
    if line != None:
        data = data[data['Route Name']==line]
    starting_location = (data['Latitude'].iloc[0],data['Longitude'].iloc[0])
    mapped = folium.Map(tiles='OpenStreetMap', location=starting_location, zoom_start=11)

    isoGeoJsonRetriever(params_iso,data,client)
    isoVisualizer(mapped,data)
    return mapped,data

In [9]:
#method that uses input dataframe with iso. Returns Map of isochrones.
def isoMapper(data,icon = 'train'):
    ## Set up folium map
    starting_location = (data['Latitude'].iloc[0],data['Longitude'].iloc[0])
    mapped = folium.Map(tiles='OpenStreetMap', location=starting_location, zoom_start=13)

    ##Add legend to Map
    #colormap = linear.RdYlBu_08.scale(station_stats[field_to_color_by].quantile(0.05),
                                     # station_stats[field_to_color_by].quantile(0.95))

    #converts Dataframe to Dictionary, necessary for the subsequent functions
    isoVisualizer(mapped,data,icon)

    return mapped

In [10]:
data_all = pd.read_csv('resources/data/klang_valley_stations_isochrones.csv')

selected_data = data_all[data_all['City']=='Kuala Lumpur']

selected_data = selected_data[(selected_data['Route Name'].isin(['Ampang Line']))]

selected_map = isoMapper(selected_data)


Done!


In [19]:
print(selected_data['iso'][0])

{"type": "FeatureCollection", "bbox": [101.687526, 3.176238, 101.701856, 3.19394], "features": [{"type": "Feature", "properties": {"group_index": 0, "value": 300.0, "center": [101.69540263355309, 3.1858327450971533], "area": 159328.79, "reachfactor": 0.2921, "total_pop": 847.0}, "geometry": {"coordinates": [[[101.694422, 3.182262], [101.694768, 3.182163], [101.695253, 3.182599], [101.69655, 3.183927], [101.697077, 3.185471], [101.697091, 3.188183], [101.695523, 3.18929], [101.694812, 3.189394], [101.694543, 3.189372], [101.694573, 3.189013], [101.695008, 3.184095], [101.694663, 3.18299], [101.694556, 3.182731], [101.694489, 3.182496], [101.694422, 3.182262]]], "type": "Polygon"}}, {"type": "Feature", "properties": {"group_index": 0, "value": 600.0, "center": [101.69540263355309, 3.1858327450971533], "area": 980240.62, "reachfactor": 0.4493, "total_pop": 7814.0}, "geometry": {"coordinates": [[[101.691158, 3.181704], [101.691202, 3.181347], [101.692238, 3.180321], [101.693147, 3.179841],

In [18]:
selected_map

In [34]:
selected_data['Longitude'][0]

3.185821

In [40]:
style_function = lambda x: {'color': '#4ef500' if x['properties']['value']<400 else ('#2100f5' if x['properties']['value']<700.0 else '#f50000'),
                                'fillOpacity' :0.25,
                                'weight':2,
                                'fillColor' :'#92daf0'}

temp = eval(station_kl_dict[200]['iso']) 
print(temp)
folium.features.GeoJson(temp,style_function = style_function).add_to(mapped) # Add GeoJson to map
mapped

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'group_index': 0, 'value': 300.0, 'center': [101.60161015012037, 3.0652656055878724], 'area': 324775.13, 'reachfactor': 0.5955, 'total_pop': 1338.0}, 'geometry': {'coordinates': [[[101.597829, 3.065154], [101.598128, 3.064722], [101.600167, 3.062499], [101.60115, 3.06212], [101.60151, 3.062118], [101.605368, 3.065222], [101.605349, 3.065581], [101.605265, 3.06568], [101.603415, 3.067457], [101.602689, 3.068106], [101.602515, 3.068166], [101.602195, 3.068175], [101.59784, 3.065514], [101.597829, 3.065154]]], 'type': 'Polygon'}, 'id': '0'}, {'type': 'Feature', 'properties': {'group_index': 0, 'value': 600.0, 'center': [101.60161015012037, 3.0652656055878724], 'area': 1189454.82, 'reachfactor': 0.5452, 'total_pop': 6902.0}, 'geometry': {'coordinates': [[[101.594105, 3.065529], [101.594111, 3.065169], [101.594147, 3.065114], [101.594671, 3.064349], [101.596278, 3.063028], [101.600531, 3.061459], [101.602098, 3.06